Reference:
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/contrib/distributions/python/ops/distribution.py
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/contrib/distributions/python/ops/distribution_util.py
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/contrib/distributions/python/ops/bernoulli.py
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/contrib/distributions/python/ops/categorical.py

## Notes.
- generic class `Distribution` have `sample(self, sample_shape=(), seed=None)` method.
- the `sample` method refers `_call_sample_n(self, sample_shape, seed, name)`.
- the `_call_sample_n` refers `_sample_n(self, n, seed)` => unique for each distribution!
- `_sample_n` refers `self.probs`.
- `self.probs` is defined when initializing each distribution class using `distribution_util.get_logits_and_probs`.

In [1]:
import tensorflow as tf
from tensorflow.contrib.distributions.python.ops import *

In [2]:
sess = tf.InteractiveSession()

In [3]:
logits = tf.constant([1.,2.,3.,4.])

## distribution_util.get_logits_and_probs

### if prob is None:
- Convert logits => probabilities

```
if multidimensional:
    return logits, tf.nn.softmax(logits)
else:
    return logits, tf.sigmoid(logits)
```     

In [4]:
logits_probs = distribution_util.get_logits_and_probs(logits=logits)
logits_probs

(<tf.Tensor 'Const:0' shape=(4,) dtype=float32>,
 <tf.Tensor 'get_logits_and_probs/probs:0' shape=(4,) dtype=float32>)

In [5]:
logits_probs[0].eval()

array([ 1.,  2.,  3.,  4.], dtype=float32)

In [6]:
logits_probs[1].eval()

array([ 0.7310586 ,  0.88079703,  0.95257413,  0.98201376], dtype=float32)

In [7]:
probs = logits_probs[1]

### if logit is None:
- Convert probabilities => logits
- inverse of sigmoid(x) = log(x) - log(1-x) = log(x) - log(1+ -1*x)

```
if multidimensional:
    return tf.log(probs), probs
else:
    return tf.log(probs) - tf.log1p(-1 * probs), probs
```

In [8]:
logits_probs = distribution_util.get_logits_and_probs(probs=probs)
logits_probs

(<tf.Tensor 'get_logits_and_probs_1/logits/sub:0' shape=(4,) dtype=float32>,
 <tf.Tensor 'get_logits_and_probs/probs:0' shape=(4,) dtype=float32>)

In [9]:
logits_probs[0].eval()

array([ 1.00000012,  1.99999952,  3.00000024,  3.99999857], dtype=float32)

In [10]:
logits_probs[1].eval()

array([ 0.7310586 ,  0.88079703,  0.95257413,  0.98201376], dtype=float32)

## Sampling examples

### 1. Bernoulli Distribution

In [11]:
bernoulli_sampler = bernoulli.Bernoulli(logits=logits)

In [12]:
bernoulli_sampler.probs.eval()

array([ 0.7310586 ,  0.88079703,  0.95257413,  0.98201376], dtype=float32)

#### At each sampling, i-th element is 1 with probability of probs[i]

In [13]:
bernoulli_sampler.sample().eval()

array([1, 1, 1, 1], dtype=int32)

#### At each sampling, output is 1 with probability of 0.3

In [14]:
bernoulli.Bernoulli(probs=0.3).sample().eval()

0

#### Calculate KL divergence

In [15]:
a_dist = bernoulli.Bernoulli(probs=0.2)
b_dist = bernoulli.Bernoulli(probs=0.7)

In [16]:
bernoulli._kl_bernoulli_bernoulli(a_dist, b_dist).eval()

0.53411078

### 2. Categorical Distribution

In [17]:
categorical_sampler = categorical.Categorical(logits=logits)

In [18]:
categorical_sampler.probs.eval()

array([ 0.0320586 ,  0.08714432,  0.23688281,  0.64391422], dtype=float32)

#### At each sampling, i-th element is 1 with probability of probs[i]

In [19]:
categorical_sampler.sample().eval()

3

#### At each sampling, output is selected with probability of `probs`

In [20]:
categorical_dist = categorical.Categorical(probs=[0.1, 0.3, 0.6])
categorical_dist.probs.eval()

array([ 0.1       ,  0.30000001,  0.60000002], dtype=float32)

In [21]:
categorical_dist.sample().eval()

2